In [1]:
import os, sys
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import datetime
import time

# df_db

In [5]:
df_db = pd.read_csv("./database/2015-12-31 00_00_00.csv", index_col=[0])
df_db = df_db.drop(columns=['cong']).reset_index(drop=True)

In [6]:
# 2호선 내선/외선 이외의 분기 노선 모두 삭제 
line2_ex_list = ['까치산', '신정네거리', '양천구청', '도림천', '용답', '신답', '용두', '신설동']
cond = df_db.loc[:, ['from_stt', 'to_stt']].applymap(lambda x: x in line2_ex_list)
cond = cond.sum(axis=1)
cond = cond > 0

drop_idx = df_db.loc[(df_db['line']=='2호선') & cond, :].index
df_db = df_db.drop(index=drop_idx)


In [7]:
# to_stt가 nan 인 것 모두 제거 
df_db = df_db.dropna()

# order를 상선/하선 표시로 변경 (후에 direction col이름을 order로 바꾸자... )
# 상선: True [1] 서울 -> 시청 / [2] (내선) 시청 -> 을지로 / [3] 오금 -> 경찰병원 / [4] 남태령 -> 사당
df_db.loc[:, 'order'] = True


In [8]:
df_db = df_db.drop_duplicates()
df_db = df_db.reset_index(drop=True)
df_db.to_csv("./final_db/df_db.csv")

# 이 때 언급되는 역의 개수: 104개
# len(set(list(df_db['from_stt']) + list(df_db['to_stt'])))

# df_holiday

In [4]:
# 공휴일 데이터 생성
# 특일 정보, REST : https://data.go.kr/iim/api/selectAPIAcountView.do

import requests, datetime
from bs4 import BeautifulSoup
import urllib.parse as urlparse

mykey = "XQ%2Fg3zsaeNMIeRAd%2ByL47ZUmTCccmTZQzEA9koPI8%2FQivAjS%2BQbL1kpCrvL0TrG84I5Yr6duLzsYPiWDqoOyPQ%3D%3D"

df_holiday = pd.DataFrame(columns=["locdate", "datename"])
for year in range(2015, 2020):
    for month in range(1,13):
        month_str = f"{month:02}"

        url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
        operation = 'getRestDeInfo'
        #operation = 'get24DivisionsInfo'
        params = {'solYear':year, 'solMonth':month_str}

        params = urlparse.urlencode(params)
        request_query = f'{url}/{operation}?{params}&serviceKey={mykey}'
        get_data = requests.get(request_query)

        if get_data.ok == True:
            soup = BeautifulSoup(get_data.content, 'html.parser')        

            item = soup.findAll('item')

            for i in item:
                df_holiday = df_holiday.append({'locdate': i.locdate.string, 'datename':i.datename.string}, ignore_index=True)
                # 요일 정보를 가지고 싶으면 이렇게.... 
                # day = int(i.locdate.string[-2:])
                # dayname = datetime.date(int(year), int(month), day).weekday()
                #weekname = get_dayname(int(year), int(month), day)
                #print(i.datename.string, i.isholiday.string, i.locdate.string,)# weekname)
df_holiday.loc[:, 'locdate'] = df_holiday.loc[:, 'locdate'].apply(lambda x: datetime.datetime.strptime(x, "%Y%m%d"))
df_holiday.to_csv("./database/holiday_2015_2019.csv")

In [9]:
df_holiday = pd.read_csv("./database/holiday_2015_2019.csv", index_col=[0])
df_holiday.loc[:, 'locdate'] = df_holiday.loc[:, 'locdate'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
df_holiday


,locdate,datename
0,2015-01-01,신정
1,2015-02-18,설날
2,2015-02-19,설날
3,2015-02-20,설날
4,2015-03-01,삼일절
...,...,...
81,2019-09-13,추석
82,2019-09-14,추석
83,2019-10-03,개천절
84,2019-10-09,한글날


In [10]:
st_t = time.time()
df_db = pd.read_csv(f"./final_db/df_db.csv", index_col=[0])

df_db.loc[:, 'date'] = df_db.loc[:, 'date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
df_db['holiday'] = '0'


# 토요일(5), 일요일(6)
print(f"Sat/Sun Start: {time.time()-st_t}")
cond_ss = df_db['date'].apply(lambda x: x.weekday())
df_db.loc[cond_ss == 5, 'holiday'] = '1'
df_db.loc[cond_ss == 6, 'holiday'] = '2'


# 공휴일
print(f"Holiday Start: {time.time()-st_t}")
cond_hd = df_db['date'].apply(lambda x: x in list(df_holiday['locdate']))
df_db.loc[cond_hd, 'holiday'] = '2'

print(f"End: {time.time()-st_t}")

df_db['holiday'].value_counts()/20


Sat/Sun Start: 7.292919158935547
Holiday Start: 10.50733494758606
End: 81.86347389221191


0    27610.0
2     7260.0
1     5280.0
Name: holiday, dtype: float64

In [11]:
df_db = df_db.drop_duplicates()
df_db = df_db.reset_index(drop=True)
df_db.to_csv("./final_db/df_db_h.csv")

In [12]:
df_db.loc[df_db['line'] == '3호선'].head(30)

,line,order,from_stt,to_stt,date,time,holiday
52,3호선,True,오금,경찰병원,2015-01-01,5,2
53,3호선,True,경찰병원,가락시장,2015-01-01,5,2
54,3호선,True,가락시장,수서,2015-01-01,5,2
55,3호선,True,수서,일원,2015-01-01,5,2
56,3호선,True,일원,대청,2015-01-01,5,2
57,3호선,True,대청,학여울,2015-01-01,5,2
58,3호선,True,학여울,대치,2015-01-01,5,2
59,3호선,True,대치,도곡,2015-01-01,5,2
60,3호선,True,도곡,매봉,2015-01-01,5,2
61,3호선,True,매봉,양재,2015-01-01,5,2
